# **Lexical and Semantic Search with Elasticsearch**

In this example, you will explore various approaches to retrieving information using Elasticsearch, focusing specifically on text, lexical and semantic search.

To accomplish this, this example demonstrates various search scenarios on a dataset generated to simulate e-commerce product information.

This dataset contains over 2,500 products, each with a description. These products are categorized into 76 distinct product categories, with each category containing a varying number of products.

## **🧰 Requirements**

For this example, you will need:

- Python 3.11 or later
- The Elastic Python client
- Elastic 9.0 deployment or later, with 8GB memory machine learning node


We'll be using [Elastic Cloud](https://www.elastic.co/guide/en/cloud/current/ec-getting-started.html), a [free trial](https://cloud.elastic.co/registration?onboarding_token=vectorsearch&utm_source=github&utm_content=elasticsearch-labs-notebook) is available.

## Setup Elasticsearch environment:

To get started, we'll need to connect to our Elastic deployment using the Python client.

Because we're using an Elastic Cloud deployment, we'll use the **Cloud Endpoint** and **Cloud API Key** to identify our deployment.


In [147]:
%pip install elasticsearch

70582.50s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [148]:
from elasticsearch import (
    Elasticsearch,
    helpers,
)  # Import the Elasticsearch client and helpers module
from urllib.request import urlopen  # library for opening URLs
import json  # module for handling JSON data
from pathlib import Path  # module for working with file paths

import getpass  # handling password input

Now we can instantiate the Python Elasticsearch client.

First we prompt the user for their password and Cloud ID.

🔐 NOTE: `getpass` enables us to securely prompt the user for credentials without echoing them to the terminal, or storing it in memory.

Then we create a `client` object that instantiates an instance of the `Elasticsearch` class.

In [151]:
# Found in the 'Manage Deployment' page
ELASTIC_ENDPOINT = getpass.getpass("Enter Elastic Endpoint:  ")

# Password for the 'elastic' user generated by Elasticsearch
ELASTIC_API_KEY = getpass.getpass("Enter Elastic API Key:  ")

# Create the client instance
client = Elasticsearch(
    hosts=[ELASTIC_ENDPOINT], api_key=ELASTIC_API_KEY, request_timeout=3600
)

Let's verify that our client is connected.

In [ ]:
resp = client.info()
print(resp)

## Setup emebdding model

Next we upload the all-mpnet-base-v2 embedding model into Elasticsearch and create an ingest pipeline with inference processors for text embedding and text expansion, using the description field for both. This field contains the description of each product.

In [ ]:
# set the model to .multilingual-e5-small-elasticsearch
es_model_id = ".multilingual-e5-small"
es_model_endpoint = ".multilingual-e5-small-elasticsearch"

# verify the model is loaded, deployed, and ready to use
models = client.ml.get_trained_models()
for model in models["trained_model_configs"]:
    if model["model_id"] == es_model_id:
        print(f"Model ID: {model['model_id']}")
        print(f"Description: {model.get('description', 'No description')}")
        print(f"Version: {model.get('version', 'N/A')}")
        break
else:
    print(f"Model {es_model_id} not found.")

print("------")

inference_endpoint = client.inference.get(inference_id=es_model_endpoint)
inference_endpoint = inference_endpoint["endpoints"][0]
print(f"Inference Endpoint ID: {es_model_endpoint}")
print(
    f"Model ID: {inference_endpoint.get('service_settings', {}).get('model_id', 'N/A')}"
)
print(f"Task Type: {inference_endpoint['task_type']}")

In [ ]:
# Creating an ingest pipeline with inference processors to use ELSER (sparse) and e5_multilingual_small (dense) to infer against data that will be ingested in the pipeline.
client.ingest.put_pipeline(
    id="ecommerce-pipeline",
    processors=[
        {
            "inference": {
                "model_id": ".elser-2-elasticsearch",
                "input_output": [
                    {
                        "input_field": "description",
                        "output_field": "elser_description_vector",
                    }
                ],
            }
        },
        {
            "inference": {
                "model_id": ".multilingual-e5-small-elasticsearch",  # Inference endpoint ID
                "input_output": [
                    {
                        "input_field": "description",
                        "output_field": "e5_description_vector",
                    }
                ],
                "inference_config": {"text_embedding": {}},
            }
        },
    ],
)

## Index documents

Then, we create a source index to load `products-ecommerce.json` or our remote source. This will be the `ecommerce` index and a destination index to extract the documents from the source and index these documents into the destination `ecommerce-search`.

For the `ecommerce-search` index we add a field to support dense vector storage and search `e5_description_vector`, this is the target field for inference results. The field type in this case is `dense_vector`, the `e5_multilingual_small` model has embedding_size of 384, so dims is set to 384. We also add a `elser_description_vector` field type to support the text expansion output.

In [ ]:
# Index to load products-ecommerce.json docs
if client.indices.exists(index="ecommerce"):
    client.indices.delete(index="ecommerce")

client.indices.create(
    index="ecommerce",
    mappings={
        "properties": {
            "product": {
                "type": "text",
            },
            "description": {
                "type": "text",
            },
            "category": {
                "type": "text",
            },
        }
    },
)

In [ ]:
# Reindex dest index

INDEX = "ecommerce-search"
if client.indices.exists(index=INDEX):
    client.indices.delete(index=INDEX)
client.indices.create(
    index=INDEX,
    mappings={
        # Saving disk space by excluding the ELSER tokens and the dense_vector field from document source.
        # Note: That should only be applied if you are certain that reindexing will not be required in the future.
        "properties": {
            "product": {
                "type": "text",
            },
            "description": {
                "type": "text",
            },
            "category": {
                "type": "text",
            },
            "elser_description_vector": {"type": "sparse_vector"},
            "e5_description_vector": {  # Inference results field, target_field.predicted_value
                "type": "dense_vector",
                "dims": 384,  # The all-mpnet-base-v2 model has embedding_size of 768, so dims is set to 768.
                "index": "true",
                "similarity": "cosine",  #  When indexing vectors for approximate kNN search, you need to specify the similarity function for comparing the vectors.
            },
        },
    },
)

## Load documents

Then we load `products-ecommerce.json` into the `ecommerce` index.

In [ ]:
#  dataset

import json

with open("products-ecommerce.json", "r") as f:
    data_json = json.load(f)


def create_index_body(doc):
    """Generate the body for an Elasticsearch document."""
    return {
        "_index": "ecommerce",
        "_source": doc,
    }


# Prepare the documents to be indexed
documents = [create_index_body(doc) for doc in data_json]

# Use helpers.bulk to index
helpers.bulk(client, documents)

print("Done indexing documents into `ecommerce` index")

## Reindex

Now we can reindex data from the `source` index `ecommerce` to the `dest` index `ecommerce-search` with the ingest pipeline `ecommerce-pipeline` we created.

After this step our `dest` index will have the fields we need to perform Semantic Search.

In [ ]:
# Reindex data from one index 'source' to another 'dest' with the 'ecommerce-pipeline' pipeline.

client.reindex(
    wait_for_completion=True,
    source={"index": "ecommerce"},
    dest={"index": "ecommerce-search", "pipeline": "ecommerce-pipeline"},
)

## Text Analysis with Standard Analyzer

In [ ]:
# Performs text analysis on a string and returns the resulting tokens.

# Define the text to be analyzed
text = "Comfortable furniture for a large balcony"

# Define the analyze request
request_body = {"analyzer": "standard", "text": text}  # Standard Analyzer

# Perform the analyze request
response = client.indices.analyze(
    analyzer=request_body["analyzer"], text=request_body["text"]
)

# Extract and display the analyzed tokens
tokens = [token["token"] for token in response["tokens"]]
print("Analyzed Tokens:", tokens)

## Text Analysis with Stop Analyzer

In [ ]:
# Performs text analysis on a string and returns the resulting tokens.

# Define the text to be analyzed
text = "Comfortable furniture for a large balcony"

# Define the analyze request
request_body = {"analyzer": "stop", "text": text}  # Stop Analyzer

# Perform the analyze request
response = client.indices.analyze(
    analyzer=request_body["analyzer"], text=request_body["text"]
)

# Extract and display the analyzed tokens
tokens = [token["token"] for token in response["tokens"]]
print("Analyzed Tokens:", tokens)

## Lexical Search

In [ ]:
# BM25

response = client.search(
    size=2,
    index="ecommerce-search",
    query={
        "match": {
            "description": {
                "query": "Comfortable furniture for a large balcony",
                "analyzer": "stop",
            }
        }
    },
)
hits = response["hits"]["hits"]

if not hits:
    print("No matches found")
else:
    for hit in hits:
        score = hit["_score"]
        product = hit["_source"]["product"]
        category = hit["_source"]["category"]
        description = hit["_source"]["description"]
        print(
            f"\nScore: {score}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n"
        )

## Semantic Search with Dense Vector

In [ ]:
# KNN

response = client.search(
    index="ecommerce-search",
    size=2,
    knn={
        "field": "e5_description_vector",
        "k": 50,  # Number of nearest neighbors to return as top hits.
        "num_candidates": 500,  # Number of nearest neighbor candidates to consider per shard. Increasing num_candidates tends to improve the accuracy of the final k results.
        "query_vector_builder": {  # Object indicating how to build a query_vector. kNN search enables you to perform semantic search by using a previously deployed text embedding model.
            "text_embedding": {
                "model_id": ".multilingual-e5-small-elasticsearch",  # Text embedding model id
                "model_text": "Comfortable furniture for a large balcony",  # Query
            }
        },
    },
)

for hit in response["hits"]["hits"]:

    score = hit["_score"]
    product = hit["_source"]["product"]
    category = hit["_source"]["category"]
    description = hit["_source"]["description"]
    print(
        f"\nScore: {score}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n"
    )

## Semantic Search with Sparse Vector

In [ ]:
# Elastic Learned Sparse Encoder - ELSER

response = client.search(
    index="ecommerce-search",
    size=2,
    query={
        "sparse_vector": {
            "field": "elser_description_vector",
            "inference_id": ".elser-2-elasticsearch",
            "query": "Comfortable furniture for a large balcony",
        }
    },
)

for hit in response["hits"]["hits"]:

    score = hit["_score"]
    product = hit["_source"]["product"]
    category = hit["_source"]["category"]
    description = hit["_source"]["description"]
    print(
        f"\nScore: {score}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n"
    )

## Hybrid Search - BM25+KNN linear combination

In [ ]:
# BM25 + KNN (Linear Combination)

response = client.search(
    index="ecommerce-search",
    size=2,
    query={
        "bool": {
            "should": [
                {
                    "match": {
                        "description": {
                            "query": "A dining table and comfortable chairs for a large balcony",
                            "boost": 1,  # You can adjust the boost value
                        }
                    }
                }
            ]
        }
    },
    knn={
        "field": "e5_description_vector",
        "k": 2,
        "num_candidates": 20,
        "boost": 1,  # You can adjust the boost value
        "query_vector_builder": {
            "text_embedding": {
                "model_id": ".multilingual-e5-small-elasticsearch",
                "model_text": "A dining table and comfortable chairs for a large balcony",
            }
        },
    },
)

for hit in response["hits"]["hits"]:

    score = hit["_score"]
    product = hit["_source"]["product"]
    category = hit["_source"]["category"]
    description = hit["_source"]["description"]
    print(
        f"\nScore: {score}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n"
    )

## Hybrid Search - BM25+KNN RRF

In [ ]:
# BM25 + KNN (RRF)
top_k = 2
response = client.search(
    index="ecommerce-search",
    retriever={
        "rrf": {
            "retrievers": [
                {
                    "standard": {
                        "query": {
                            "match": {
                                "description": "A dining table and comfortable chairs for a large balcony"
                            }
                        }
                    }
                },
                {
                    "knn": {
                        "field": "e5_description_vector",
                        "query_vector_builder": {
                            "text_embedding": {
                                "model_id": ".multilingual-e5-small",
                                "model_text": "A dining table and comfortable chairs for a large balcony",
                            }
                        },
                        "k": 2,
                        "num_candidates": 20,
                    }
                },
            ],
            "rank_window_size": 2,
            "rank_constant": 20,
        }
    },
)

for hit in response["hits"]["hits"]:

    score = hit["_score"]
    category = hit["_source"]["category"]
    product = hit["_source"]["product"]
    description = hit["_source"]["description"]
    print(
        f"Score: {score}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n"
    )

## Hybrid Search - BM25+ELSER linear combination

In [ ]:
# BM25 + Elastic Learned Sparse Encoder (Linear Combination)

response = client.search(
    index="ecommerce-search",
    size=2,
    query={
        "bool": {
            "should": [
                {
                    "match": {
                        "description": {
                            "query": "A dining table and comfortable chairs for a large balcony",
                            "boost": 1,  # You can adjust the boost value
                        }
                    }
                },
                {
                    "sparse_vector": {
                        "field": "elser_description_vector",
                        "inference_id": ".elser-2-elasticsearch",
                        "query": "A dining table and comfortable chairs for a large balcony",
                    }
                },
            ]
        }
    },
)

for hit in response["hits"]["hits"]:

    score = hit["_score"]
    product = hit["_source"]["product"]
    category = hit["_source"]["category"]
    description = hit["_source"]["description"]
    print(
        f"\nScore: {score}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n"
    )

## Hybrid Search - BM25+ELSER RRF

In [ ]:
# BM25 + ELSER (RRF)
top_k = 2
response = client.search(
    index="ecommerce-search",
    retriever={
        "rrf": {
            "retrievers": [
                {
                    "standard": {
                        "query": {
                            "match": {
                                "description": "A dining table and comfortable chairs for a large balcony"
                            }
                        }
                    }
                },
                {
                    "standard": {
                        "query": {
                            "sparse_vector": {
                                "field": "elser_description_vector",
                                "inference_id": ".elser-2-elasticsearch",
                                "query": "A dining table and comfortable chairs for a large balcony",
                            }
                        }
                    }
                },
            ],
            "rank_window_size": 2,
            "rank_constant": 20,
        }
    },
)

for hit in response["hits"]["hits"]:

    score = hit["_score"]
    category = hit["_source"]["category"]
    product = hit["_source"]["product"]
    description = hit["_source"]["description"]
    print(
        f"Score: {score}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n"
    )